In [2]:
from uuid import uuid3, NAMESPACE_DNS

In [38]:
str(uuid3(NAMESPACE_DNS, repr(LGBMClassifier())))[-6:]

'a6e579'

In [16]:
from hashlib import md5

In [42]:
md5(repr(LGBMClassifier(n_estimators=3)).encode("utf-8")).hexdigest()[-8:]

'c950be60'

In [59]:
str(type(LGBMClassifier())).split(".")

["<class 'lightgbm", 'sklearn', "LGBMClassifier'>"]

In [58]:
str(LGBMClassifier().__class__)

"<class 'lightgbm.sklearn.LGBMClassifier'>"

In [47]:
from src.util import get_obj_ref

In [60]:
get_obj_ref(LGBMClassifier())

'LGBMClassifier_ad4f6715'

In [1]:
import pandas as pd

In [50]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.util import set_context, comp_path

In [4]:
from src.models import run_estimator_cv, PARAMS_SKOPT, get_oof_predictions

In [10]:
from skopt import BayesSearchCV
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_predict

In [6]:
set_context("titanic")

Files in data directory:
______

titanic/
    clean.pkl
    raw/
        gender_submission.csv
        test.csv
        train.csv
______



In [7]:
df = pd.read_pickle(comp_path("clean.pkl"))
df.shape

(1309, 13)

In [8]:
tr = df[~df._test]
tr.shape

(891, 13)

In [9]:
tr.sample(2)

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,_test
654,18.0,NaN,Q,6.75,"Hegarty, Miss. Hanora ""Nora""",0,655,3,female,0,0.0,365226,False
692,NaN,NaN,S,56.50,"Lam, Mr. Ali",0,693,3,male,0,1.0,1601,False


In [10]:
target = "Survived"
X_cols = set(tr.columns) - {target}

In [11]:
X = tr[X_cols]
y = tr[target]

In [12]:
est = LGBMClassifier()

In [13]:
est.fit(X, y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [14]:
PARAMS_SKOPT["lgb_small_trees"]

{'n_estimators': Integer(low=50, high=2000),
 'max_depth': Integer(low=1, high=8),
 'num_leaves': Integer(low=4, high=32),
 'learning_rate': Real(low=0.0001, high=10, prior='log-uniform', transform='identity')}

In [15]:
est.best_score_

defaultdict(dict, {})

In [20]:
est = BayesSearchCV(
    LGBMClassifier(),
    PARAMS_SKOPT["lgb_big_trees"],
    scoring="roc_auc",
    cv=5,
    n_iter=25,
    n_jobs=-1,
    verbose=1,
)

In [21]:
est.fit(X, y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.6s finished


BayesSearchCV(cv=5, error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid=True, n_iter=25, n_jobs=-1, n_points=1,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
       refit=True, return_train_score=False, scoring='roc_auc',
       search_spaces={'n_estimators': Integer(low=5, high=500), 'max_depth': Integer(low=100, high=200), 'num_leaves': Integer(low=50, high=500), 'learning_rate': Real(low=0.0001, high=10, prior='log-uniform', transform='identity')},
       verbose=1)

In [22]:
repr(est.best_estimator_)

"LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,\n        importance_type='split', learning_rate=0.02135085801759193,\n        max_depth=185, min_child_samples=20, min_child_weight=0.001,\n        min_split_gain=0.0, n_estimators=111, n_jobs=-1, num_leaves=270,\n        objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,\n        silent=True, subsample=1.0, subsample_for_bin=200000,\n        subsample_freq=0)"

In [23]:
est.best_score_

0.8751528408995995

In [25]:
pd.DataFrame(est.cv_results_)

,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,param_num_leaves,params
0,0.737352,0.833992,0.847259,0.847059,0.890853,0.831134,0.050790,1,0.273891,0.051536,0.035035,0.007026,0.857423,150,363,247,"{'learning_rate': 0.8574231231790369, 'max_dep..."
1,0.822200,0.811265,0.858222,0.851604,0.893551,0.847248,0.028981,1,0.121674,0.009035,0.028192,0.005294,0.000114,192,55,365,"{'learning_rate': 0.00011414761496439964, 'max..."
2,0.822266,0.812648,0.858088,0.850802,0.901309,0.848893,0.031141,1,0.138442,0.020374,0.038130,0.013516,0.000728,153,67,245,"{'learning_rate': 0.0007278521408165658, 'max_..."
3,0.676943,0.828722,0.872259,0.862032,0.898813,0.827506,0.078758,1,0.154218,0.026141,0.055299,0.022090,1.008750,129,90,489,"{'learning_rate': 1.008749959584565, 'max_dept..."
4,0.843017,0.824901,0.890374,0.871056,0.914463,0.868633,0.032076,1,0.276985,0.070125,0.038770,0.011122,0.030460,166,146,285,"{'learning_rate': 0.0304598447170795, 'max_dep..."
5,0.553689,0.607115,0.456417,0.646925,0.718564,0.596369,0.088198,1,0.066077,0.010396,0.025852,0.004749,3.056280,160,106,113,"{'learning_rate': 3.056280460373948, 'max_dept..."
6,0.832740,0.827734,0.869519,0.880080,0.897126,0.861330,0.027003,1,0.410341,0.088831,0.034837,0.007479,0.002376,143,266,308,"{'learning_rate': 0.0023759017639191486, 'max_..."
7,0.734783,0.577668,0.562834,0.578877,0.585267,0.608020,0.063982,1,0.084643,0.014303,0.029246,0.000977,3.326033,199,182,296,"{'learning_rate': 3.3260326189311176, 'max_dep..."
8,0.836364,0.821476,0.867580,0.876270,0.895035,0.859237,0.026796,1,0.172679,0.036308,0.050907,0.017002,0.004570,102,105,152,"{'learning_rate': 0.004570304197891879, 'max_d..."
9,0.698353,0.636298,0.709492,0.655749,0.614949,0.663032,0.035991,1,0.142236,0.020973,0.041028,0.018038,4.724942,142,475,376,"{'learning_rate': 4.724941679498092, 'max_dept..."


In [26]:
cross_val_predict(est.best_estimator_, X, y, method="predict_proba")[:, 1]

C:\Users\Liam\Anaconda3\envs\kaggle\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


array([0.06427451, 0.90985372, 0.30625736, 0.85428916, 0.11371714,
       0.06081841, 0.11671647, 0.3914611 , 0.32082449, 0.91344565,
       0.81357712, 0.83355016, 0.07548558, 0.0785559 , 0.69627998,
       0.79279278, 0.31660328, 0.0571257 , 0.53793957, 0.69908025,
       0.07179157, 0.07816423, 0.48915564, 0.17888923, 0.10037492,
       0.1614398 , 0.05980076, 0.17204782, 0.69908025, 0.04793298,
       0.24800219, 0.92060041, 0.69908025, 0.09879008, 0.23334793,
       0.14906571, 0.05980076, 0.07548558, 0.60035827, 0.50607611,
       0.31721814, 0.79822668, 0.06402198, 0.91317755, 0.53679976,
       0.07623034, 0.0552934 , 0.69908025, 0.06711605, 0.60035827,
       0.31660328, 0.08398018, 0.91172312, 0.79668138, 0.13581097,
       0.08396178, 0.80956815, 0.12650701, 0.86783623, 0.31660328,
       0.09866948, 0.90985372, 0.19113065, 0.31660328, 0.12596769,
       0.11966863, 0.80808882, 0.06588729, 0.40252421, 0.04750199,
       0.08960677, 0.0965991 , 0.11170035, 0.07385849, 0.13731